# Dates and Timestamps

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("dates").getOrCreate()

In [ ]:
df = spark.read.csv("../datasets/appl_stock.csv",header=True,inferSchema=True)

In [ ]:
df.show(5, truncate=False)

Let's walk through how to grab parts of the timestamp data

In [ ]:
from pyspark.sql.functions import format_number,dayofmonth,hour,dayofyear,month,year,weekofyear,date_format

In [ ]:
df.select(dayofmonth(df['Date'])).show()

In [ ]:
df.select(hour(df['Date'])).show()

In [ ]:
df.select(dayofyear(df['Date'])).show()

In [ ]:
df.select(month(df['Date'])).show()

So for example, let's say we wanted to know the average closing price per year. Easy! With a groupby and the year() function call:

In [ ]:
df.select(year(df['Date'])).show(5, truncate=False)

In [ ]:
df.withColumn("Year",year(df['Date'])).show(5, truncate=False)

In [ ]:
newdf = df.withColumn("Year",year(df['Date']))

In [ ]:
newdf.show(5, truncate=False)

In [ ]:
import pyspark.sql.functions as F

In [ ]:
newdf.select(F.current_date().alias("current_date"), F.current_timestamp().alias("current_timestamp")).show(truncate=False) # YYYY-mm-dd # YYYY-mm-dd HH:MM:SS.ssssss

In [ ]:
max_high = newdf.select(F.max("High").alias("high"), F.min("High").alias("low")).collect()

In [ ]:
type(max_high)

In [ ]:
max_high[0]

In [ ]:
max_high[0]["high"], max_high[0]["low"]

In [ ]:
newdf.select("year").distinct().sort("year").show()

In [ ]:
columns = newdf.columns

In [ ]:
needed_columns = columns[:4]

In [ ]:
needed_columns

In [ ]:
newdf.select(*needed_columns).show()

In [ ]:
newdf.select(newdf.columns).show()

In [ ]:
df1 = spark.createDataFrame([('2022-08-16',)], ['dt'])

In [ ]:
df1.show()

In [ ]:
df1.select(date_format('dt', 'MM/dd/yyyy').alias('date')).collect()

In [ ]:
newdf.groupBy("Year").max()[["Year", "max(High)"]].show(truncate=False)

In [ ]:
newdf.groupBy("Year").mean()[['avg(Year)','avg(Close)']].show()

In [ ]:
result = newdf.groupBy("Year").mean()[['avg(Year)','avg(Close)']]

In [ ]:
result = result.withColumnRenamed("avg(Year)","Year")

In [ ]:
result = result.select('Year',format_number('avg(Close)',2).alias("Mean Close")).show()

In [ ]:
spark.stop()